In [171]:
# Import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [172]:
# import dataset
df= pd.read_csv('nova_pay_combined.csv')
df.head(3)

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,...,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,fee8542d-8ee6-4b0d-9671-c294dd08ed26,402cccc9-28de-45b3-9af7-cc5302aa1f93,2022-10-03 18:40:59.468549+00:00,US,USD,CAD,ATM,278.19,278.19,4.25,...,0.123,standard,263,0.522,0,0.223,0,0,0.0,0
1,bfdb9fc1-27fe-4a85-b043-4d813d679259,67c2c6b3-ef0a-4777-a3f1-c84a851bb6ad,2022-10-03 20:39:38.468549+00:00,CA,CAD,MXN,web,208.51,154.29,4.24,...,0.569,standard,947,0.475,0,0.268,0,1,0.0,0
2,fc855034-3ea5-4993-9afa-b511d93fe5e8,6d0d9b27-fa26-45f8-93b1-2df29d182d9c,2022-10-03 23:02:43.468549+00:00,US,USD,CNY,mobile,160.33,160.33,2.70,...,0.437,enhanced,367,0.939,0,0.176,0,0,0.0,0


In [173]:
# checking for shape
df.shape

(11400, 26)

In [174]:
# checking for missing values
df.isnull().sum()

transaction_id                 0
customer_id                    0
timestamp                     29
home_country                   0
source_currency                0
dest_currency                  0
channel                        0
amount_src                     0
amount_usd                   305
fee                          295
exchange_rate_src_to_dest      0
device_id                      0
new_device                     0
ip_address                   305
ip_country                   301
location_mismatch              0
ip_risk_score                  0
kyc_tier                     300
account_age_days               0
device_trust_score           295
chargeback_history_count       0
risk_score_internal            0
txn_velocity_1h                0
txn_velocity_24h               0
corridor_risk                  0
is_fraud                       0
dtype: int64

In [175]:
# checking for data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11400 entries, 0 to 11399
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   transaction_id             11400 non-null  object 
 1   customer_id                11400 non-null  object 
 2   timestamp                  11371 non-null  object 
 3   home_country               11400 non-null  object 
 4   source_currency            11400 non-null  object 
 5   dest_currency              11400 non-null  object 
 6   channel                    11400 non-null  object 
 7   amount_src                 11400 non-null  object 
 8   amount_usd                 11095 non-null  float64
 9   fee                        11105 non-null  float64
 10  exchange_rate_src_to_dest  11400 non-null  float64
 11  device_id                  11400 non-null  object 
 12  new_device                 11400 non-null  bool   
 13  ip_address                 11095 non-null  obj

In [176]:
# Target column is_fraud, checking for the bool analysis
df["is_fraud"].value_counts(normalize=True)

is_fraud
0    0.912544
1    0.087456
Name: proportion, dtype: float64

In [177]:
# handling timestamp data type to datetime and coercing invalid values to NaT
df["timestamp"]=pd.to_datetime(df["timestamp"], errors="coerce")


In [178]:
# convert amount_src to float from object
df["amount_src"]=pd.to_numeric(df["amount_src"], errors="coerce")

In [179]:
# Handling missing Values in amount in usd
exchange_rates = df[df["amount_usd"].notna()].groupby("source_currency").apply(
    lambda x: (x["amount_usd"] / x["amount_src"]).mean()).to_dict()


C:\Users\khali\AppData\Local\Temp\ipykernel_14008\2616436385.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exchange_rates = df[df["amount_usd"].notna()].groupby("source_currency").apply(


In [180]:
exchange_rates


{'CAD': 0.7216095926871465,
 'GBP': 1.223441221648679,
 'USD': 0.9838730321259439}

In [181]:
df["source_currency"].value_counts()

source_currency
USD    8031
GBP    2139
CAD    1230
Name: count, dtype: int64

In [182]:
df["amount_usd"] = df.apply(
    lambda row: row["amount_usd"] if pd.notna(row["amount_usd"]) 
    else row["amount_src"] * exchange_rates.get(row["source_currency"],1), axis=1
)

In [183]:
#Handling the fee missing value using median.
if "fee"in df.columns:
    if "channel" in df.columns:
        df["fee"]=df.groupby("channel")["fee"].transform(lambda s: s.fillna(s.median()))
        df["fee"].fillna(df["fee"].median())



In [184]:
# Handling ip_country missing values
if {"ip_country","home_country"} .issubset (df.columns):
    df["ip_country"]=df["ip_country"].fillna(df["home_country"])

In [185]:
#Handling kyc_tier mising value 
# kyc_tier: fill with Mode 
if'kyc_tier' in df.columns:
   if 'kyc_tier' in df.columns:
    mode_kyc=df['kyc_tier'].mode().iloc[0] if not df['kyc_tier'].mode().empty else 'standard'
    df['kyc_tier']=df['kyc_tier'].fillna(mode_kyc)

In [186]:
# Handle missing device_trust_score values

if 'device_trust_score' in df.columns:
    if {'new_device', 'kyc_tier'}.issubset(df.columns):

        df['device_trust_score'] = (
            df
            .groupby(['new_device', 'kyc_tier'])['device_trust_score']
            .transform(lambda s: s.fillna(s.median()))
        )

    # Fallback: fill any remaining missing values with global median
    df['device_trust_score'] = df['device_trust_score'].fillna(
        df['device_trust_score'].median()
    )


C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\khali\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [187]:
df.isna().sum()

transaction_id                 0
customer_id                    0
timestamp                     61
home_country                   0
source_currency                0
dest_currency                  0
channel                        0
amount_src                     4
amount_usd                     0
fee                            0
exchange_rate_src_to_dest      0
device_id                      0
new_device                     0
ip_address                   305
ip_country                     0
location_mismatch              0
ip_risk_score                  0
kyc_tier                       0
account_age_days               0
device_trust_score             0
chargeback_history_count       0
risk_score_internal            0
txn_velocity_1h                0
txn_velocity_24h               0
corridor_risk                  0
is_fraud                       0
dtype: int64

In [188]:
# dropping the remaining missing values as timestamp and amount_src could not be converted
# and ip_address as a unique location
df.dropna(inplace=True)

In [189]:
#Sanity checks for the fraud dection datasets and describing will give an idea of the numerical values 
df.describe(include="all")

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,...,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
count,11030,11030,11030,11030,11030,11030,11030,11030.000000,11030.000000,11030.000000,...,11030.000000,11030,11030.000000,11030.000000,11030.000000,11030.000000,11030.000000,11030.000000,11030.000000,11030.000000
unique,10836,1314,NaN,7,3,9,12,NaN,NaN,NaN,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,d1849b01-2aef-47af-9363-6eb542a11203,402cccc9-28de-45b3-9af7-cc5302aa1f93,NaN,US,USD,NGN,mobile,NaN,NaN,NaN,...,NaN,standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,1458,NaN,7684,7770,1426,6134,NaN,NaN,NaN,...,NaN,7883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2024-05-03 10:31:41.290405376+00:00,NaN,NaN,NaN,NaN,437.800284,448.664391,98.222628,...,0.397463,NaN,393.040435,0.654513,0.050045,0.268070,0.470263,0.740073,0.045573,0.089483
min,NaN,NaN,2022-10-03 18:40:59.468549+00:00,NaN,NaN,NaN,NaN,-9997.160000,7.230000,-1.000000,...,0.004000,NaN,1.000000,-0.100000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,2023-08-15 02:53:40.468549120+00:00,NaN,NaN,NaN,NaN,90.867500,92.545000,2.390000,...,0.209000,NaN,147.000000,0.515000,0.000000,0.169000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,2024-05-09 01:16:28.968549120+00:00,NaN,NaN,NaN,NaN,159.080000,163.590000,3.510000,...,0.325000,NaN,286.500000,0.658000,0.000000,0.223000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,2025-01-29 07:26:17.047345408+00:00,NaN,NaN,NaN,NaN,294.940000,302.137500,5.550000,...,0.488000,NaN,661.000000,0.894000,0.000000,0.391000,0.000000,0.000000,0.050000,0.000000
max,NaN,NaN,2025-12-16 00:13:41.468549+00:00,NaN,NaN,NaN,NaN,11942.890000,12497.900000,9999.990000,...,1.200000,NaN,1095.000000,0.999000,2.000000,0.900000,8.000000,11.000000,0.250000,1.000000


In [190]:
# count negative values in key numeric columns 
neg_counts={
    "amount_src":(df["amount_src"]<0).sum(),
    "amount_usd":(df["amount_usd"]<0).sum(),
    "fee":(df["fee"]<0).sum(),
    "device_trust_score":(df["device_trust_score"]<0).sum(),
    "txn_velocity_1h":(df["txn_velocity_1h"]<0).sum(),
    "txn_velocity_24h":(df["txn_velocity_24h"]<0).sum(),
    "risk_score_internal":(df["risk_score_internal"]<0).sum(),
}
neg_counts

{'amount_src': np.int64(100),
 'amount_usd': np.int64(0),
 'fee': np.int64(90),
 'device_trust_score': np.int64(190),
 'txn_velocity_1h': np.int64(190),
 'txn_velocity_24h': np.int64(0),
 'risk_score_internal': np.int64(0)}

In [191]:
# Remove the negative counts
df= df[
    (df["amount_src"]>= 0) &
       (df["fee"]>= 0) &
       (df["device_trust_score"]>= 0) &
        (df["txn_velocity_1h"]>= 0) 
      
]

In [192]:
(df["amount_usd"] / df["amount_src"]).describe()

count    10840.000000
mean         1.018123
std          0.136521
min          0.739788
25%          1.000000
50%          1.000000
75%          1.000000
max          1.250405
dtype: float64

In [193]:
#Check for future time_stamp
df[df["timestamp"] > pd.Timestamp.utcnow()]

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,...,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud


In [194]:
df["location_mismatch"].value_counts()

location_mismatch
False    9047
True     1793
Name: count, dtype: int64

In [195]:
#check for unique vaue count for cat features
df["channel"].unique()
df["source_currency"].unique()
df["dest_currency"].unique()
df["kyc_tier"].unique()

array(['standard', 'enhanced', 'low', ' standard  ', 'standrd',
       ' enhanced  ', 'STANDARD', 'unknown', 'enhancd', ' low  ',
       'ENHANCED', 'LOW'], dtype=object)

In [196]:
# check for each unique cat 
df["channel"].unique()

array(['ATM', 'web', 'mobile', 'WEB', ' web  ', 'MOBILE', 'unknown',
       'mobille', ' mobile  ', 'weeb', 'ATm', ' ATM  '], dtype=object)

In [197]:
#solving the unique cat to be unique
df["channel"]=df["channel"].str.lower().str.strip()

In [198]:
df["channel"] = df["channel"].replace({
    "web": "web",
    "weeb": "web",

    "mobile": "mobile" , 
    "mobille": "mobile",

    "atm": "atm",


})

In [199]:
df["channel"] = df["channel"].replace({"unknown": np.nan})

In [200]:
# check for each unique cat 
df["source_currency"].unique()

array(['USD', 'CAD', 'GBP'], dtype=object)

In [201]:
# check for each unique cat 
df["dest_currency"].unique()

array(['CAD', 'MXN', 'CNY', 'EUR', 'INR', 'GBP', 'PHP', 'NGN', 'USD'],
      dtype=object)

In [202]:
# check for each unique cat 
df["kyc_tier"].unique()

array(['standard', 'enhanced', 'low', ' standard  ', 'standrd',
       ' enhanced  ', 'STANDARD', 'unknown', 'enhancd', ' low  ',
       'ENHANCED', 'LOW'], dtype=object)

In [203]:
df["kyc_tier"]=df["kyc_tier"].str.lower().str.strip()

In [204]:
#sovle unique cat to smal letters and correct the cat
df["kyc_tier"] = df["kyc_tier"].replace({
    "enhanced": "enhanced",
    "enhancd": "enhanced",
   
    

    "standard": "standard" ,
    "standrd": "standard",
   

     "low":"low",
    
   
   

})

In [205]:
df["kyc_tier"] = df["kyc_tier"].replace({"unknown": np.nan})

In [206]:
df.isna().sum()

transaction_id                0
customer_id                   0
timestamp                     0
home_country                  0
source_currency               0
dest_currency                 0
channel                      36
amount_src                    0
amount_usd                    0
fee                           0
exchange_rate_src_to_dest     0
device_id                     0
new_device                    0
ip_address                    0
ip_country                    0
location_mismatch             0
ip_risk_score                 0
kyc_tier                     29
account_age_days              0
device_trust_score            0
chargeback_history_count      0
risk_score_internal           0
txn_velocity_1h               0
txn_velocity_24h              0
corridor_risk                 0
is_fraud                      0
dtype: int64

In [207]:
df.dropna(inplace=True)

In [208]:
df.to_csv("cleaned_transactons.csv", index=False)
print("Cleaned dataset saved as'cleaned_transacton.csv'")


Cleaned dataset saved as'cleaned_transacton.csv'
